# Citations

How can we get a model to cite which parts of the source documents it referenced in its response?

To explore some techniques for extracting citations, let's first create a simple RAG chain. To start we'll just retrieve from the web using the [ExaRetriever](https://js.langchain.com/docs/integrations/retrievers/exa).

## Setup
### Dependencies

We’ll use an OpenAI chat model and embeddings and a Memory vector store in this walkthrough, but everything shown here works with any [ChatModel](/docs/modules/model_io/chat) or [LLM](/docs/modules/model_io/llms), [Embeddings](https://js.langchain.com/docs/modules/data_connection/text_embedding/), and [VectorStore](https://js.langchain.com/docs/modules/data_connection/vectorstores/) or [Retriever](/docs/modules/data_connection/retrievers/).

We’ll use the following packages:

```bash
npm install --save langchain @langchain/exa @langchain/openai cheerio
```

We need to set environment variables for Exa & OpenAI:

```bash
export OPENAI_API_KEY=YOUR_KEY
export EXASEARCH_API_KEY=YOUR_KEY
```

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with [LangSmith](https://smith.langchain.com/).

Note that LangSmith is not needed, but it is helpful. If you do want to use LangSmith, after you sign up at the link above, make sure to set your environment variables to start logging traces:


```bash
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=YOUR_KEY
```

In [1]:
import Exa from "exa-js";
import { ExaRetriever } from "@langchain/exa";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const exaClient = new Exa();
const llm = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});
const search = new ExaRetriever({
  client: exaClient,
});
const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You're a helpful AI assistant. Given a user question and some Wikipedia article snippets, answer the user question. If none of the articles answer the question, just say you don't know.\n\nHere are the Wikipedia articles:{context}"],
  ["human", "{question}"],
])

Now that we've got a model, retriever and prompt, let's chain them all together. We'll need to add some logic for formatting our retrieved `Document`s to a string that can be passed to our prompt. We'll make it so our chain returns both the answer and the retrieved Documents.

In [ ]:
import { Document } from "@langchain/core/documents";

/**
 * Get the page content of the documents, and format into a string.
 */
const formatDocs = async (docs: Array<Document>): Promise<string> => {
  const results = await exaClient.getContents(docs.map((doc) => doc.metadata.id));
  const formatted = docs.map((doc, idx) => `Article title: ${doc.metadata.title}\nArticle Snippet: ${results[idx].}`);
}